In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from __future__ import division

from copy import deepcopy
import pickle
import time
import os
import uuid

import scipy
import numpy as np

from matplotlib import pyplot as plt
import matplotlib as mpl

from mimi import envs
from mimi import utils
from mimi import user_models
from mimi import opt
from mimi import reward_models
from mimi import models

In [ ]:
data_dir = os.path.join(utils.data_dir, 'isql')

In [ ]:
sess = utils.make_tf_session(gpu_mode=False)

In [ ]:
env = envs.ISQLEnv()

In [ ]:
pilot_ids = [
  'spike',
  'jet',
  'faye',
  'vicious',
  'ed',
  'ein',
  'julia',
  'punch',
  'judy',
  'lin',
  'grencia',
  'laughingbull'
]

In [ ]:
isql_dir = os.path.join(data_dir, 'raw')

In [ ]:
newton_data_dir = os.path.join(isql_dir, '5.1-lander-newton')
aristotle_data_dir = os.path.join(isql_dir, '5.1-lander-aristotle')

In [ ]:
newton_rollouts_of_pilot = {k: [] for k in pilot_ids}
aristotle_rollouts_of_pilot = {k: [] for k in pilot_ids}

In [ ]:
for pilot_id in pilot_ids:
  with open(os.path.join(newton_data_dir, '%s_pilot_policy_demo_rollouts.pkl' % pilot_id), 'rb') as f:
    newton_rollouts_of_pilot[pilot_id].extend(sum(pickle.load(f), []))
  with open(os.path.join(aristotle_data_dir, '%s_pilot_policy_demo_rollouts.pkl' % pilot_id), 'rb') as f:
    aristotle_rollouts_of_pilot[pilot_id].extend(sum(pickle.load(f), []))

In [ ]:
def format_obs(s, a):
  return np.concatenate((s[:-1], a))

def format_ep(rollout):
  ep = []
  for x in rollout:
    a = utils.onehot_encode(x[1], env.n_act_dim)
    r = x[2]
    s = format_obs(x[0], a)
    ns = format_obs(x[3], a)
    ep.append((s, a, r, ns, False, {}))
  return ep

def format_eps(rollouts):
  return [format_ep(ep) for ep in rollouts]

In [ ]:
methods = ['aristotle', 'newton']
rollouts_of_pol = []
method_of_pol = []
pilot_of_pol = []
for method in methods:
  for pilot_id in pilot_ids:
    rollouts = eval('%s_rollouts_of_pilot' % method)[pilot_id]
    eps = format_eps(rollouts)
    rollouts_of_pol.append(eps)
    method_of_pol.append(method)
    pilot_of_pol.append(pilot_id)

In [ ]:
n_conds = len(rollouts_of_pol)
n_steps = sum(len(r) for x in rollouts_of_pol for r in x)
n_steps, n_conds, n_steps / n_conds

In [ ]:
mi_model_init_args = [sess]
mi_model_init_kwargs = {
  'n_env_obs_dim': env.n_env_obs_dim,
  'n_user_obs_dim': env.n_user_obs_dim,
  'n_act_dim': env.n_act_dim,
  'n_layers': 2,
  'layer_size': 64
}
mi_model_train_kwargs = {
  'iterations': 1000,
  'ftol': 1e-6,
  'learning_rate': 1e-4,
  'batch_size': 64,
  'val_update_freq': None,
  'verbose': False,
  'warm_start': False
}

In [ ]:
reward_model = reward_models.MIRewardModel(
  env,
  mi_model_init_args,
  mi_model_init_kwargs,
  mi_model_train_kwargs
)

In [ ]:
ixs_reward_model = reward_models.MIRewardModel(
  env,
  mi_model_init_args,
  mi_model_init_kwargs,
  mi_model_train_kwargs,
  use_next_env_obs=False
)

In [ ]:
true_rew_of_rollout = lambda rollout: np.mean([x[2] for x in rollout])
true_reward_model = lambda rollouts: np.mean([true_rew_of_rollout(rollout) for rollout in rollouts])

In [ ]:
offline_reward_models = [true_reward_model, reward_model, ixs_reward_model]

In [ ]:
n_seeds = 1
rewards_of_pol = np.zeros((n_seeds, len(rollouts_of_pol), len(offline_reward_models)))

In [ ]:
for i in range(n_seeds):
  rewards_of_pol[i, :, :] = utils.compute_rews_of_rollouts(
    rollouts_of_pol,
    offline_reward_models,
    verbose=True
  )

In [ ]:
rewards_of_pol_path = os.path.join(data_dir, 'rewards_of_pol.pkl')

In [ ]:
with open(rewards_of_pol_path, 'wb') as f:
  pickle.dump(rewards_of_pol, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(rewards_of_pol_path, 'rb') as f:
  rewards_of_pol = pickle.load(f)

In [ ]:
rewards_of_pol[:, :, 1:] = np.maximum(rewards_of_pol[:, :, 1:], 0)

In [ ]:
mean_rewards_of_pol = np.mean(rewards_of_pol, axis=0)
mean_rewards_of_pol

In [ ]:
mean_rewards_of_pol[:, 2] = np.minimum(mean_rewards_of_pol[:, 2], mean_rewards_of_pol[:, 1])
mean_rewards_of_pol[:, 2] = np.maximum(mean_rewards_of_pol[:, 2], 0)
mean_rewards_of_pol = np.concatenate((mean_rewards_of_pol, (mean_rewards_of_pol[:, 1] - mean_rewards_of_pol[:, 2])[:, np.newaxis]), axis=1)
mean_rewards_of_pol

In [ ]:
label_of_method = {
  'newton': 'Solo Human',
  'aristotle': 'With Copilot',
}
color_of_method = {
  'newton': 'gray',
  'aristotle': 'orange',
}
color_of_pol = [color_of_method[m] for m in method_of_pol]

In [ ]:
idxes_of_method = {m: [] for m in methods}
for i, method in enumerate(method_of_pol):
  idxes_of_method[method].append(i)

In [ ]:
mpl.rcParams.update({'font.size': 12})

In [ ]:
corr = scipy.stats.spearmanr(mean_rewards_of_pol)
corr

In [ ]:
rho = corr[0][0, 1]
rho

In [ ]:
plt.title(r'Internal-to-Real Dynamics Transfer ($\rho$ = %0.2f)' % rho)
plt.xlabel('True Reward')
plt.ylabel(r"$\mathcal{I}(\mathbf{x}_t, (\mathbf{s}_t, \mathbf{s}_{t+1}))$")
for method, idxes in idxes_of_method.items():
  plt.scatter(
    mean_rewards_of_pol[idxes, 0], 
    mean_rewards_of_pol[idxes, 1],
    color=color_of_method[method],
    label=label_of_method[method],
    s=50
  )
plt.legend(loc='best')
plt.savefig(os.path.join(data_dir, 'isql-offline-eval-truerew-vs-mi-per-poluser.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
rewards_of_method = [[[] for _ in range(mean_rewards_of_pol.shape[1])] for _ in methods]
idx_of_method = {x: i for i, x in enumerate(methods)}
for i, method in enumerate(method_of_pol):
  for j in range(mean_rewards_of_pol.shape[1]):
    rewards_of_method[idx_of_method[method]][j].append(mean_rewards_of_pol[i, j])
rewards_of_method = [[np.mean(x) for x in y] for y in rewards_of_method]
rewards_of_method = np.array(rewards_of_method)

In [ ]:
plt.title('Internal-to-Real Dynamics Transfer')
plt.xlabel('True Reward')
plt.ylabel(r"$\mathcal{I}(\mathbf{x}_t, (\mathbf{s}_t, \mathbf{s}_{t+1}))$")
for i, method in enumerate(methods):
  plt.scatter(
    rewards_of_method[i, 0], 
    rewards_of_method[i, 1],
    color=color_of_method[method],
    label=label_of_method[method],
    s=50
  )
plt.legend(loc='best')
plt.savefig(os.path.join(data_dir, 'isql-offline-eval-truerew-vs-mi-per-pol.pdf'), bbox_inches='tight')
plt.show()